In [1]:
! pip install datasets evaluate transformers sentencepiece nltk
! pip install datsets transformers[sentencepiece]
! pip install -U accelerate
! pip install -U transformers
# ! pip install -U torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset, DatasetDict, Dataset, load_metric
from evaluate import load

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [5]:
import transformers

print(transformers.__version__)

4.35.2


# Fine-tuning a model on a Question Generation task

In [6]:
model_checkpoint = "google/mt5-small"

## Loading the dataset

In [7]:
import json
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset, load_metric

def load_data(file):

  # Specify the path to your JSON file
  json_file_path = file
  # Initialize an empty dictionary
  data = {}

  # Load the JSON data into the dictionary
  try:
      with open(json_file_path, "r") as json_file:
          data = json.load(json_file)
      print("JSON file loaded successfully into a dictionary.")
  except FileNotFoundError:
      print(f"Error: JSON file '{json_file_path}' not found.")
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
  except Exception as e:
      print(f"An error occurred: {e}")

  return data

df_train = load_data( '/content/drive/MyDrive/IRE/datasets/final_mt5_dataset.json')

JSON file loaded successfully into a dictionary.


In [8]:
data_finetune_mixed = Dataset.from_dict(df_train)
print(data_finetune_mixed)

print(data_finetune_mixed['input_ids'][0])
print(data_finetune_mixed['input_ids'][8])
print(data_finetune_mixed['input_ids'][17])

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 9000
})
[250100, 259, 267, 486, 1668, 481, 6273, 351, 71394, 549, 7305, 7785, 23022, 263, 32213, 527, 287, 62181, 807, 48461, 270, 11395, 281, 287, 4836, 332, 259, 81421, 259, 69737, 260, 486, 4248, 9629, 22407, 351, 4265, 157663, 438, 259, 70277, 261, 259, 262, 837, 325, 74767, 22285, 259, 48970, 304, 287, 2279, 13749, 261, 1070, 2554, 285, 32213, 351, 2476, 17577, 2602, 304, 28901, 533, 27906, 9192, 52342, 5370, 288, 287, 13992, 305, 90865, 270, 304, 22049, 6540, 304, 259, 81421, 259, 69737, 52764, 285, 281, 631, 120249, 7670, 455, 287, 13749, 260, 259, 39609, 288, 259, 262, 1068, 1668, 481, 9454, 2584, 110386, 32944, 261, 259, 129416, 304, 32213, 39796, 42441, 345, 259, 66337, 287, 20862, 19208, 1421, 332, 259, 5480, 1866, 259, 15176, 7400, 296, 259, 139262, 259, 66337, 87023, 64860, 263, 305, 259, 66127, 4368, 304, 31521, 332, 287, 2508, 305, 186071, 20099, 296, 305, 259, 119999, 2478, 4788, 1009, 259, 262, 2985, 1866, 

In [9]:
# !pip install rouge-score
# raw_datasets = load_dataset("xsum")
squad_validation = load_dataset("squad", split="validation")
# metric = load("rouge")
metric = load_metric('bleu')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

<ipython-input-9-66974704cca5>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('bleu')


In [10]:
squad_validation

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 10570
})

To access an actual element, you need to select a split first, then give an index:

In [11]:
squad_validation[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [12]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [13]:
documents = show_random_elements(squad_validation)
documents

,id,title,context,question,answers
0,56d9c660dc89441400fdb7e8,Super_Bowl_50,"After each team punted, Panthers quarterback Cam Newton appeared to complete a 24-yard pass Jerricho Cotchery, but the call was ruled an incompletion and upheld after a replay challenge. CBS analyst and retired referee Mike Carey stated he disagreed with the call and felt the review clearly showed the pass was complete. A few plays later, on 3rd-and-10 from the 15-yard line, linebacker Von Miller knocked the ball out of Newton's hands while sacking him, and Malik Jackson recovered it in the end zone for a Broncos touchdown, giving the team a 10–0 lead. This was the first fumble return touchdown in a Super Bowl since Super Bowl XXVIII at the end of the 1993 season.",Who sacked Newton a few plays after the challenge?,"{'text': ['Von Miller', 'Von Miller', 'Miller'], 'answer_start': [389, 389, 393]}"
1,571cdcb85efbb31900334e0e,Oxygen,"Among the most important classes of organic compounds that contain oxygen are (where ""R"" is an organic group): alcohols (R-OH); ethers (R-O-R); ketones (R-CO-R); aldehydes (R-CO-H); carboxylic acids (R-COOH); esters (R-COO-R); acid anhydrides (R-CO-O-CO-R); and amides (R-C(O)-NR\n2). There are many important organic solvents that contain oxygen, including: acetone, methanol, ethanol, isopropanol, furan, THF, diethyl ether, dioxane, ethyl acetate, DMF, DMSO, acetic acid, and formic acid. Acetone ((CH\n3)\n2CO) and phenol (C\n6H\n5OH) are used as feeder materials in the synthesis of many different substances. Other important organic compounds that contain oxygen are: glycerol, formaldehyde, glutaraldehyde, citric acid, acetic anhydride, and acetamide. Epoxides are ethers in which the oxygen atom is part of a ring of three atoms.",What function do compounds like phenol and acetone serve in the manufacture of many other substances?,"{'text': ['feeder materials', 'as feeder materials', 'feeder materials', 'feeder materials', 'as feeder materials'], 'answer_start': [546, 543, 546, 546, 543]}"
2,5725cfd0271a42140099d228,"Fresno,_California","In 1872, the Central Pacific Railroad established a station near Easterby's—by now a hugely productive wheat farm—for its new Southern Pacific line. Soon there was a store around the station and the store grew the town of Fresno Station, later called Fresno. Many Millerton residents, drawn by the convenience of the railroad and worried about flooding, moved to the new community. Fresno became an incorporated city in 1885. By 1931 the Fresno Traction Company operated 47 streetcars over 49 miles of track.",How many streetcars did the Fresno Traction Company operate in 1931?,"{'text': ['47 streetcars', '47', '47'], 'answer_start': [471, 471, 471]}"
3,57302e45947a6a140053d232,Islamism,"In its focus on the Caliphate, the party takes a different view of Muslim history than some other Islamists such as Muhammad Qutb. HT sees Islam's pivotal turning point as occurring not with the death of Ali, or one of the other four rightly guided Caliphs in the 7th century, but with the abolition of the Ottoman Caliphate in 1924. This is believed to have ended the true Islamic system, something for which it blames ""the disbelieving (Kafir) colonial powers"" working through Turkish modernist Mustafa Kemal Atatürk.",What are colonial powers blamed for?,"{'text': ['ended the true Islamic system', 'working through Turkish modernist Mustafa Kemal Atatürk', 'abolition of the Ottoman Caliphate'], 'answer_start': [359, 463, 290]}"
4,57273e50dd62a815002e9a04,Construction,"Engineering News-Record (ENR) is a trade magazine for the construction industry. Each year, ENR compiles and reports on data about the size of design and construction companies. They publish a list of the largest companies in the United States (Top-40) and also a list the largest global firms (Top-250, by amount of work they are doing outside their home country). In 2014, ENR compiled the data in nine market segments. It was divided as transportation, 


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [14]:
metric

Metric(name: "bleu", features: {'predictions': Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), 'references': Sequence(feature=Sequence(feature=Value(dtype='string', id='token'), length=-1, id='sequence'), length=-1, id='references')}, usage: """
Computes BLEU score of translated segments against one or more references.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
Returns:
    'bleu': bleu score,
    'precisions': geometric mean of n-gram precisions,
    'brevity_penalty': brevity penalty,
    'length_ratio': ratio of lengths,
    'translation_length': translation_length,
    'reference_length': reference_length
Examples

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [15]:
# fake_preds = [["how"], ["are"], "question is good?"]
# fake_labels = ["how is he?", "is this question good?"]
# metric.compute(predictions=fake_preds, references=fake_labels)

## Preprocessing the data

In [16]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) t5-small
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) #mt5-small
# tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

len_tokenizer =len(tokenizer) # 32100 to get the sentinel ids
print(f"len_tokenizer = {len_tokenizer}")

# # adding 'questiongeneration' as token
# tokenizer.add_tokens(['questiongeneration'])
# print(tokenizer.convert_tokens_to_ids('questiongeneration'))

# len_tokenizer =len(tokenizer) # 32100 to get the sentinel ids
# print(f"len_tokenizer = {len_tokenizer}")

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


len_tokenizer = 250100


In [17]:
tokenizer("Hello, this one sentence!")
# {'input_ids': [30273, 261, 714, 1371, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


{'input_ids': [30273, 261, 714, 1371, 259, 98923, 309, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])
# {'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the targets for our model, we need to tokenize them using the `text_target` parameter. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [19]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[30273, 261, 714, 1371, 259, 98923, 309, 1], [1494, 339, 259, 7845, 259, 98923, 260, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [21]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = ['questiongeneration: ' + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["question"], max_length=max_target_length, truncation=True)
    # labels = tokenizer(labels, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
# max_input_length = 1024
# max_target_length = 128

# def preprocess_function(examples):
#     inputs = ['summarize: ' + doc for doc in examples["document"]]
#     model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

#     # Setup the tokenizer for targets
#     labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [23]:
preprocess_function(squad_validation[:2])

{'input_ids': [[7680, 96542, 267, 3309, 59413, 1013, 639, 461, 6369, 28920, 3416, 288, 259, 28905, 287, 69624, 304, 287, 5139, 29526, 10667, 274, 48441, 271, 332, 287, 814, 11989, 260, 486, 6369, 29526, 26701, 274, 111991, 271, 69624, 259, 70444, 169024, 337, 269, 62956, 345, 287, 5139, 29526, 26701, 274, 99699, 271, 69624, 25087, 259, 89693, 263, 840, 158206, 288, 259, 45563, 259, 1616, 259, 18211, 3309, 59413, 10331, 260, 486, 3416, 639, 259, 41866, 351, 259, 4293, 6469, 654, 261, 344, 40064, 277, 263, 48547, 281, 287, 1716, 14598, 5596, 15146, 344, 5909, 259, 85014, 261, 16352, 260, 1477, 714, 639, 287, 1013, 807, 3309, 59413, 261, 287, 259, 41245, 259, 265, 170086, 285, 287, 313, 159618, 259, 110300, 311, 514, 259, 17250, 17088, 264, 235549, 259, 51356, 263, 261, 527, 2857, 527, 26011, 137382, 39136, 347, 287, 17562, 304, 2135, 347, 259, 3151, 3309, 59413, 3416, 514, 10462, 25595, 5155, 274, 15403, 259, 1542, 287, 3416, 259, 2220, 783, 2101, 259, 15965, 527, 313, 28062, 59413, 458,

In [24]:
def reduce_data(raw_datasets, no_of_rows):

  shuffled_dataset = raw_datasets.shuffle(seed=42)
  reduced_data = shuffled_dataset.select(range(no_of_rows))

  return reduced_data
  # new_dataset_dict = {
  #     "train": shuffled_dataset["train"].select(range(x_train)),
  #     "validation": shuffled_dataset["validation"].select(range(x_val)),
  #     # "test": shuffled_dataset["test"].select(range(x_test)),
  # }

  # new_dataset_dict = DatasetDict(new_dataset_dict)
  # return new_dataset_dict

reduced_dataset = reduce_data(squad_validation, 720)

In [25]:
reduced_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 720
})

In [26]:
tokenized_squad_validation = reduced_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

In [27]:
show_random_elements(tokenized_squad_validation)

,id,title,context,question,answers,input_ids,attention_mask,labels
0,57274118dd62a815002e9a1f,American_Broadcasting_Company,"Warner tried with mixed success to adapt some of its most successful films as ABC television series, and showcase these adaptations as part of the wheel series Warner Bros. Presents. Airing during the 1955–56 season, it showcased television adaptations of the 1942 films Kings Row and Casablanca; Cheyenne (adapted from the 1947 film Wyoming Kid); Sugarfoot (a remake of the 1954 film The Boy from Oklahoma); and Maverick. However, the most iconic of ABC's relationships with Hollywood producers was its agreement with Walt Disney; after the start of the network's bond with the Disney studio, James Lewis Baughman, who worked as a columnist at that time, observed that ""at ABC's headquarters in New York, the secretaries [were now] wearing hats with Mickey Mouse ears"".",In what series did ABC present it's 1950s film adaptations in?,"{'text': ['Warner Bros. Presents', 'wheel series Warner Bros. Presents', 'wheel series'], 'answer_start': [160, 147, 147]}","[7680, 96542, 267, 259, 91173, 259, 31157, 514, 259, 80232, 10067, 288, 12192, 2155, 304, 2476, 2250, 259, 30344, 39291, 527, 37116, 60643, 12614, 261, 305, 259, 263, 134068, 259, 3824, 12192, 6540, 527, 2456, 304, 287, 57007, 12614, 259, 91173, 74453, 260, 30703, 263, 260, 1889, 347, 259, 6768, 287, 55913, 1326, 5037, 11989, 261, 609, 259, 263, 134068, 285, 60643, 12192, 6540, 304, 287, 52251, 39291, 54514, 43372, 305, 259, 138548, 296, 8159, 113013, 274, 122700, 345, 702, 287, 50460, 1766, 5743, 107600, 41661, 853, 62088, 47014, 274, 262, 584, 25167, 304, 287, 53817, 1766, 486, 13863, 702, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[563, 2119, 12614, 3031, 37116, 4963, 609, 277, 263, 31634, 263, 1766, 12192, 6540, 281, 291, 1]"
1,56e7550700c9c71400d76fc7,Teacher,"Perhaps the most significant difference between primary school and secondary school teaching is the relationship between teachers and children. In primary schools each class has a teacher who stays with them for most of the week and will teach them the whole curriculum. In secondary schools they will be taught by different subject specialists each session during the week and may have ten or more different teachers. The relationship between children and their teachers tends to be closer in the primary school where they act as form tutor, specialist teacher and surrogate parent during the course of the day.",What does a teacher teach in primary school?,"{'text': ['the whole curriculum', 'whole curriculum', 'whole curriculum', 'the whole curriculum'], 'answer_start': [249, 253, 253, 249]}","[7680, 96542, 267, 1118, 34606, 287, 2250, 259, 20364, 25297, 259, 4964, 259, 22139, 5560, 305, 259, 52912, 5560, 259, 45037, 339, 287, 31713, 259, 4964, 259, 66071, 305, 10295, 260, 563, 259, 22139, 259, 53971, 259, 3151, 3931, 1070, 259, 262, 40434, 1866, 7186, 263, 514, 2486, 332, 2250, 304, 287, 5219, 305, 898, 64692, 2486, 287, 18695, 259, 99542, 260, 563, 259, 52912, 259, 53971, 287, 276, 898, 390, 259, 270, 37124, 455, 5673, 14234, 22102, 263, 259, 3151, 35332, 259, 6768, 287, 5219, 305, 1432, 783, 3034, 631, 1097, 5673, 259, 66071, 260, 486, 31713, 259, 4964, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]","[5126, 259, 6975, 259, 262, 40434, 64692, 281, 259, 22139, 5560, 291, 1]"
2,57309564069b5314008321a8,Imperialism,"During the 20th century, historians John Gallagher (1919–1980)

## Fine-tuning the model

In [28]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MT5Model, MT5ForConditionalGeneration

# model = MT5Model.from_pretrained("google/mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
model
# model = MT5ForConditionalGeneration.from_pretrained("google/" + model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [29]:
import accelerate

accelerate.__version__

# !pip install -U torch

'0.24.1'

In [30]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
print(model_name)
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-mixed",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    # fp16=True,
    # push_to_hub=True,
)

mt5-small


In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
import nltk
nltk.download('punkt')
import numpy as np
from nltk.translate.bleu_score import corpus_bleu

def compute_metrics(eval_pred):
    # Unpack the predictions and labels from the input.
    predictions, labels = eval_pred

    # Decode the predictions to human-readable text, skipping special tokens.
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace any instance of -100 in the labels with the pad token id.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode the labels to human-readable text, skipping special tokens.
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # For BLEU, we need to tokenize the sentences into words.
    # nltk's word_tokenize function can be used for this purpose.
    tokenized_preds = [nltk.word_tokenize(pred.strip().lower()) for pred in decoded_preds]
    # BLEU expects a list of reference translations for each sentence.
    tokenized_labels = [[nltk.word_tokenize(label.strip().lower())] for label in decoded_labels]

    # Compute the BLEU score for the entire corpus (set of predictions).
    # The function corpus_bleu takes a list of list of reference translations and a list of candidate translations.
    bleu_score = corpus_bleu(tokenized_labels, tokenized_preds)

    # Add mean generated length.
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    gen_len = np.mean(prediction_lens)

    # Return BLEU score and generated length. BLEU score is already a percentage so no need to multiply by 100.
    return {"bleu": round(bleu_score, 4), "gen_len": round(gen_len, 4)}


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [33]:
tokenized_squad_validation

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 720
})

In [34]:
data_finetune_mixed

print(data_finetune_mixed['input_ids'][0])
print(data_finetune_mixed['input_ids'][8])
print(data_finetune_mixed['input_ids'][17])

[250100, 259, 267, 486, 1668, 481, 6273, 351, 71394, 549, 7305, 7785, 23022, 263, 32213, 527, 287, 62181, 807, 48461, 270, 11395, 281, 287, 4836, 332, 259, 81421, 259, 69737, 260, 486, 4248, 9629, 22407, 351, 4265, 157663, 438, 259, 70277, 261, 259, 262, 837, 325, 74767, 22285, 259, 48970, 304, 287, 2279, 13749, 261, 1070, 2554, 285, 32213, 351, 2476, 17577, 2602, 304, 28901, 533, 27906, 9192, 52342, 5370, 288, 287, 13992, 305, 90865, 270, 304, 22049, 6540, 304, 259, 81421, 259, 69737, 52764, 285, 281, 631, 120249, 7670, 455, 287, 13749, 260, 259, 39609, 288, 259, 262, 1068, 1668, 481, 9454, 2584, 110386, 32944, 261, 259, 129416, 304, 32213, 39796, 42441, 345, 259, 66337, 287, 20862, 19208, 1421, 332, 259, 5480, 1866, 259, 15176, 7400, 296, 259, 139262, 259, 66337, 87023, 64860, 263, 305, 259, 66127, 4368, 304, 31521, 332, 287, 2508, 305, 186071, 20099, 296, 305, 259, 119999, 2478, 4788, 1009, 259, 262, 2985, 1866, 91386, 263, 90161, 1472, 260, 1]
[21940, 250100, 95236, 250099, 97793, 

In [35]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=data_finetune_mixed,
    eval_dataset=tokenized_squad_validation,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
print()

We can now finetune our model by just calling the `train` method:

In [36]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,9.446000,3.117667,0.014700,10.225000
2,6.381400,2.777699,0.029300,11.825000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2250, training_loss=9.565397677951388, metrics={'train_runtime': 1228.2481, 'train_samples_per_second': 14.655, 'train_steps_per_second': 1.832, 'total_flos': 5896657861263360.0, 'train_loss': 9.565397677951388, 'epoch': 2.0})

In [37]:
import os

def save_model(model, tokenizer, model_path):
  if not os.path.exists(model_path):
    os.makedirs(model_path)
  print("Directory created for new checkpt to save")

  model.save_pretrained(model_path)
  tokenizer.save_pretrained(model_path)

model_path = '/content/drive/MyDrive/IRE/models/' + 'mt5-fintuned-mixed-dataset'
save_model(model, tokenizer, model_path)

Directory created for new checkpt to save


In [38]:
# from transformers import AutoModelForSeq2SeqLM

# model_path = '/content/drive/MyDrive/IRE/models/' + 'mt5-fintuned-mixed-dataset.pt'
# model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
# model

In [39]:
# import torch

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
# model

In [40]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def _generate(input):
  context = "questionsgeneration: " + input
  # print(context)
  model_inputs = tokenizer(context, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
  # print(model_inputs)

  output_ids = model.generate(model_inputs["input_ids"], max_length=50, num_beams=4, early_stopping=True)
  decoded_preds = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in output_ids]
  # print(decoded_preds)
  return decoded_preds


# context = "చంద్రబాబుపై ఈసీకి విజయసాయి రెడ్డి ఫిర్యాదు ఎన్నికల నియమావళి అమల్లో ఉన్నా చంద్రబాబు ప్రజావేదిక ద్వారా పార్టీ కార్యక‍్రమాలు నిర్వహిస్తున్నారని విజయసాయి రెడ్డి విమర్శిం్చారు. ప్రజావేదికలో ఎమ్మెల్యేలను పిలిచి టెలీ కాన్ఫరెన్స్‌లు, పార్టీ సమీక్షలు నిర్వహిస్తున్నారని తెలిపారు. హైదరాబాద్‌: ఆంద్రప్రదేశ్ ముఖ్యమంత్రి నారా చంద్రబాబు నాయుడిపై వైఎఎస్సార్ కాంగ్రెసు పార్టీ రాజ్యసభ సభ్యుడు విజయసాయి రెడ్డి కేంద్ర ఎన్నికల కమిషన్ కు ఆయన ఫిర్యాదు చేశారు. ఈ మేరకు ఆయన ఈసీకి ఆదివారం లేఖ రాశారు. ముఖ్యమంత్రి చంద్రబాబు యధేచ్ఛగా అధికార దుర్వినియోగానికి పాల్పడుతున్నారని ఆయన ఆ లేఖలో ఫిర్యాదు చేశారు. ఎన్నికల నియమావళి అమల్లో ఉన్నా చంద్రబాబు ప్రజావేదిక ద్వారా పార్టీ కార్యక‍్రమాలు నిర్వహిస్తున్నారని విజయసాయి రెడ్డి విమర్శిం్చారు. ప్రజావేదికలో ఎమ్మెల్యేలను పిలిచి టెలీ కాన్ఫరెన్స్‌లు, పార్టీ సమీక్షలు నిర్వహిస్తున్నారని తెలిపారు. ప్రజావేదిక ప్రభుత్వానికి సంబంధించిన భవనమని ఆయన తెలిపారు. ఎన్నికల నియమావళి అమల్లో ఉన్నప్పుడు ఇలాంటి సమావేశాలు ఈసీ అనుమతితో చేయాల్సి ఉందని ఆయన గుర్తు చేశారు. చంద్రబాబు ఈసీ అనుమతి తీసుకున్నారో లేదో తమకు తెలియదని, ఈ విషయంపై సీఈసీ వెంటనే జోక‍్యం చేసుకోవాలని ఆయన అన్నారు. విజయసాయి రెడ్డి లేఖ పాఠం ఇదే.. ప్రజా వేదిక ప్రభుత్వ సముదాయం దానిని పార్టీ అవసరాల కోసం ముఖ్యమంత్రి, మంత్రులు వాడుతున్నారు. దీంతో పాటు రాష్ట్ర ప్రభుత్వ ఆస్తి అయిన వీడియో, టెలీ కాన్ఫరెన్స్ సదుపాయాలను పార్టీ అవసరాలకి ఉపయోగిస్తున్నారు. ఎన్నికల కోడ్ అమలులో ఉండగా ఒక పార్టీ మాత్రమే ఈ సదుపాయాలను ఉపయోగించుకోవడం సమంజసం కాదు. ప్రభుత్వ అతిథి భవనాలు, మీటింగ్ హాల్‌లు, వీడియో, టెలీ కాన్ఫరెన్స్ సదుపాయాలను మిగిలిన పార్టీలకు కూడా ఉపయోగించుకోవడానికి అవకాశం కల్పించాలి. ఆయా సదుపాయాలను ఉపయోగించుకునేందుకు ముఖ్యమంత్రి ఎన్నికల సంఘం అనుమతి తీసుకున్నారా? లేదా..? మాకు తెలియజేయండి. ఇప్పటికైనా ఎన్నికల సంఘం ఈ సంఘటనలపై సమీక్షించి రాష్ట్ర ప్రధాన కార్యదర్శికి ఎన్నికల కోడ్ సక్రమంగా అమలు పరిచేలా స్పష్టమైన ఆదేశాలు ఇవండి"
# print(_generate(context))
# context = "Angels in the Outfield was Eisenhower's favorite movie. His favorite reading material for relaxation were the Western novels of Zane Grey. With his excellent memory and ability to focus, Eisenhower was skilled at card games. He learned poker, which he called his \"favorite indoor sport,\" in Abilene. Eisenhower recorded West Point classmates' poker losses for payment after graduation, and later stopped playing because his opponents resented having to pay him. A classmate reported that after learning to play contract bridge at West Point, Eisenhower played the game six nights a week for five months."
# print(_generate(context))

In [41]:
# import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def __generate(examples):
    inputs = ['questiongeneration: ' + doc for doc in examples["context"]]

    model_inputs = tokenizer(inputs, padding=True, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
    # print(model_inputs)
    output_ids = model.generate(model_inputs["input_ids"], max_length=max_target_length, num_beams=4, early_stopping=True)

    decoded_preds = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in output_ids]
    model_inputs["question"] = decoded_preds
    return model_inputs

# context = "ఉండి ఆంధ్ర మహిళాసభలో మెట్రిక్ చదివింది. రాజావారి ఆస్తిపాస్తులు కోర్టు వ్యవహారాలలో చిక్కుకుని ఆర్థిక ఒడిదుడుకులు ఏర్పడి ఈమె మళ్ళీ సినిమాలలో నటించాల్సి వచ్చింది. 1991 వరకు సినిమాలలో నటించి ప్రస్తుతం చెన్నైలో కుమారుని వద్ద నివసిస్తున్నది. నాటకరంగం. ఈమె చింతామణి, శ్రీకాళహస్తి మహాత్మ్యం, శాంతినివాసం, మనోహర, పంజరంలో పక్షులు, బాలనాగమ్మ, చాణక్య చంద్రగుప్త, శ్రీకృష్ణతులాభారం మొదలైన అనేక నాటకాలలో నటించి పేరుగడించింది. సూరవరపు వెంకటేశ్వర్లు, కాళిదాసు కోటేశ్వరరావు, పద్మనాభం, మాడా వెంకటేశ్వరరావు, స్థానం నరసింహారావు, టి.జి.కమలాదేవి, వాణిశ్రీ, జమున, ఎస్.వి.రంగారావు, శారద, రామకృష్ణ, నాగభూషణం, వల్లం నరసింహారావు, జి.వరలక్ష్మి మొదలైన నటీనటులతో కలిసి నాటకాలలో నటించింది. సినిమారంగం. ఈమె 60కి పైగా తెలుగు సినిమాలలో, నాలుగైదు తమిళ సినిమాలలో, కొన్ని కన్నడ సినిమాలలో, ఒక ఒరియా సినిమాలో నటించింది. అవకాశాలు తగ్గినప్పుడు రాజసులోచన, సుకుమారి, మంజుల, కుయిలీ, జ్యోతిలక్ష్మి, సిల్క్‌స్మిత మొదలైన వారికి తమిళం నుండి తెలుగులోనికి వచ్చిన అనేక"
# result = _generate(Dataset.from_dict({'context': context}))
# print(result["question"])

# # context = "సినిమాలకు డబ్బింగ్ చెప్పింది. ఈమె నటించిన సినిమాలలో కొన్ని: టెలివిజన్. ఈమె నాగమ్మ, పవిత్రబంధం, అక్క, మూడుముళ్ళబంధం మొదలైన తెలుగు టి.వి.సీరియళ్లలోను, శివమయం, కాలం, చిత్తి మొదలైన తమిళ టి.వి.సీరియళ్లలోను నటించింది."
# # _generate({'context': context})
# # print()

# # context = "చిత్ర కథ. చిత్ర కథ విషయానికి వస్తే, గౌతం ఒక రాక్ స్టార్. గౌతంకి మెదడుకి సంబంధిచిన (ఇంటిగ్రేషన్ డిజార్దర్) (మెదడు గుర్తు పెట్టుకునే సామర్థ్యం తక్కువగా) జబ్బు ఉంటుంది. గౌతమ్ పదేళ్ల వయసులో అతని తల్లి దండ్రులను ఎవరో చంపేయడంతో అనాథ శరణాలంలో పెరుగుతాడు. గౌతంకి అతని తల్లిదండ్రులు ఎలా ఉంటారో గుర్తు ఉండదు. గౌతం చిన్నప్పుడు కొన్ని కారణాల వల్ల అతని తల్లిదండ్రులని ముగ్గురు వ్యక్తులు కలసి చంపుతారు. వాళ్ళని చంపాలనే పగతో ఆ ముగ్గురిని గుర్తు పెట్టుకుంటాడు. కానీ గౌతమ్ అనుకుంటున్నట్టుగా అతని తల్లిదండ్రులను ఎవరూ చంపేయలేదని, తల్లిదండ్రులు లేని అనాథ అయిన గౌతం సృష్టించుకున్న ఊహల్లో మాత్రమే అతని తల్లిదండ్రులు, వారిని చంపిన హంతకులు ఉన్నారని అందరి నమ్మకం. అది నిజం కాదని, తన తల్లిదండ్రులని నిజంగానే ముగ్గురు వ్యక్తులు చంపేశారని గౌతం ఎంత చెప్పినా ఎవరూ నమ్మరు. తన నమ్మకం, ఇతరుల అపనమ్మకం మధ్య"
# # _generate({'context': context})
# # print()

# # context = "ఊగిలాడుతూ పెరిగి పెద్దవాడైన గౌతం ఒక రాక్ స్టార్ గా మంచి పేరు ప్రఖ్యాతులు సంపాదిస్తాడు. కానీ తన తల్లిదండ్రులను చంపిన ఆ ముగ్గురు మాత్రం అతన్ని వెంటాడుతూనే ఉంటారు. వాళ్ళని చంపానని అనుకుని పోలీసులకు లొంగిపోతాడు. కాని పోలీసులు అసలు విషయం తెలుసుకుని ఆశ్చర్యపోతారు. తనకు తానే వాళ్ళ అమ్మ నాన్నలని చంపిన వాడిని చంపానని చెప్పి పోలీసులకు లొంగిపోతాడు. చివరకు గౌతం వాళ్ళ అమ్మనాన్నలు ఎలా వుంటారో తెలుసుకున్నడా లేదా అన్నది అసలు కథ. ఈ సినిమా కథ వైవిధ్యంగా ఉంది. అంతే కాక అత్యధిక భారీ మొత్తంలో ఈ సినిమాని నిర్మించారు. ప్రేక్షకులు సినిమా చూస్తున్నంత సేపు ఎప్పుడు ఎం జరుగుతుందా అనిపించేలా సుకుమార్ ఈ సినిమాని రూపొందించారు. ఇంటర్నెట్ మూవీ డేటాబేస్ లో ఈ సినిమా భారతదేశసినిమాలో అత్యధికపాయింట్లు పొందిన సినిమాగా చరిత్ర సృష్టించి, భారతీయ సినిమాకే గర్వించదగ్గ సినిమాగా ఈచిత్రం నిలిచింది. ఒక ప్రముఖ"
# # _generate(context)
# # print()

# # context  = 'జీవిత విశేషాలు. ఈమె అసలు పేరు సరోజిని. ఈమె గుంటూరు జిల్లా, తెనాలి మండలం, చినరావూరు గ్రామంలో ఒక వ్యవసాయ కుటుంబంలో సూరయ్య, వెంకటరత్నమ్మ దంపతులకు జన్మించింది. ఈమెకు బాల్యం నుండే సంగీతం, నాట్యం నేర్పించారు. ఎం.ఎస్.శైవ అనే నాట్యాచార్యుని వద్ద ఈమె కూచిపూడి నాట్యం నేర్చుకుంది. ఈమె విద్యాభ్యాసం తెనాలి మునిసిపల్ హైస్కూలులో సాగింది. హైస్కూలులో ఉన్నప్పుడే ఈమె నాటకాలలో నటించడం మొదలుపెట్టింది. బేబి సుజాత &amp; పార్టీ పేరుతో ఒక డ్యాన్స్ ట్రూపు పెట్టి నాట్యప్రదర్శనలు ఇచ్చింది. ఈమె నాట్యకౌశలానికి మెచ్చి 11 ఏళ్ల అతి పిన్న వయసులోనే భువనగిరిలో నృత్య ఉపాధ్యాయురాలిగా ఉద్యోగం లభించింది. సినిమాలలో నటించాలనే కోరికతో చెన్నైకి చేరుకుని సినిమాలకు ప్రయత్నిస్తూనే నాటకాలలో నటించింది. పంజరంలో పక్షులు అనే నాటక ప్రదర్శనలో ఈమెను చూసిన పిఠాపురం రాజా గారి మనవడు ఎం.ఎం.కె.ఆప్పారావు ఇష్టపడి పెళ్ళి చేసుకున్నాడు. పెళ్ళి తరువాత ఈమె సినిమాలకు కొంతకాలం దూరంగా'
# # _generate(context)
# # print()

# # context = "చంద్రబాబుపై ఈసీకి విజయసాయి రెడ్డి ఫిర్యాదు ఎన్నికల నియమావళి అమల్లో ఉన్నా చంద్రబాబు ప్రజావేదిక ద్వారా పార్టీ కార్యక‍్రమాలు నిర్వహిస్తున్నారని విజయసాయి రెడ్డి విమర్శిం్చారు. ప్రజావేదికలో ఎమ్మెల్యేలను పిలిచి టెలీ కాన్ఫరెన్స్‌లు, పార్టీ సమీక్షలు నిర్వహిస్తున్నారని తెలిపారు. హైదరాబాద్‌: ఆంద్రప్రదేశ్ ముఖ్యమంత్రి నారా చంద్రబాబు నాయుడిపై వైఎఎస్సార్ కాంగ్రెసు పార్టీ రాజ్యసభ సభ్యుడు విజయసాయి రెడ్డి కేంద్ర ఎన్నికల కమిషన్ కు ఆయన ఫిర్యాదు చేశారు. ఈ మేరకు ఆయన ఈసీకి ఆదివారం లేఖ రాశారు. ముఖ్యమంత్రి చంద్రబాబు యధేచ్ఛగా అధికార దుర్వినియోగానికి పాల్పడుతున్నారని ఆయన ఆ లేఖలో ఫిర్యాదు చేశారు. ఎన్నికల నియమావళి అమల్లో ఉన్నా చంద్రబాబు ప్రజావేదిక ద్వారా పార్టీ కార్యక‍్రమాలు నిర్వహిస్తున్నారని విజయసాయి రెడ్డి విమర్శిం్చారు. ప్రజావేదికలో ఎమ్మెల్యేలను పిలిచి టెలీ కాన్ఫరెన్స్‌లు, పార్టీ సమీక్షలు నిర్వహిస్తున్నారని తెలిపారు. ప్రజావేదిక ప్రభుత్వానికి సంబంధించిన భవనమని ఆయన తెలిపారు. ఎన్నికల నియమావళి అమల్లో ఉన్నప్పుడు ఇలాంటి సమావేశాలు ఈసీ అనుమతితో చేయాల్సి ఉందని ఆయన గుర్తు చేశారు. చంద్రబాబు ఈసీ అనుమతి తీసుకున్నారో లేదో తమకు తెలియదని, ఈ విషయంపై సీఈసీ వెంటనే జోక‍్యం చేసుకోవాలని ఆయన అన్నారు. విజయసాయి రెడ్డి లేఖ పాఠం ఇదే.. ప్రజా వేదిక ప్రభుత్వ సముదాయం దానిని పార్టీ అవసరాల కోసం ముఖ్యమంత్రి, మంత్రులు వాడుతున్నారు. దీంతో పాటు రాష్ట్ర ప్రభుత్వ ఆస్తి అయిన వీడియో, టెలీ కాన్ఫరెన్స్ సదుపాయాలను పార్టీ అవసరాలకి ఉపయోగిస్తున్నారు. ఎన్నికల కోడ్ అమలులో ఉండగా ఒక పార్టీ మాత్రమే ఈ సదుపాయాలను ఉపయోగించుకోవడం సమంజసం కాదు. ప్రభుత్వ అతిథి భవనాలు, మీటింగ్ హాల్‌లు, వీడియో, టెలీ కాన్ఫరెన్స్ సదుపాయాలను మిగిలిన పార్టీలకు కూడా ఉపయోగించుకోవడానికి అవకాశం కల్పించాలి. ఆయా సదుపాయాలను ఉపయోగించుకునేందుకు ముఖ్యమంత్రి ఎన్నికల సంఘం అనుమతి తీసుకున్నారా? లేదా..? మాకు తెలియజేయండి. ఇప్పటికైనా ఎన్నికల సంఘం ఈ సంఘటనలపై సమీక్షించి రాష్ట్ర ప్రధాన కార్యదర్శికి ఎన్నికల కోడ్ సక్రమంగా అమలు పరిచేలా స్పష్టమైన ఆదేశాలు ఇవండి"
# # _generate(context)
# # print()

# # context = "Angels in the Outfield was Eisenhower's favorite movie. His favorite reading material for relaxation were the Western novels of Zane Grey. With his excellent memory and ability to focus, Eisenhower was skilled at card games. He learned poker, which he called his \"favorite indoor sport,\" in Abilene. Eisenhower recorded West Point classmates' poker losses for payment after graduation, and later stopped playing because his opponents resented having to pay him. A classmate reported that after learning to play contract bridge at West Point, Eisenhower played the game six nights a week for five months."
# # print(_generate(context))
# # inference = {}
# # inference["context"] = context
# # _generate(context)

In [42]:
dpr_data_path = '/content/drive/MyDrive/IRE/datasets/telugu_minidata_dpr.json'

with open(dpr_data_path, "r", encoding='utf-8') as json_file:
    df_dpr = json.load(json_file)

# df_dpr = load_data(dpr_data_path)
# print(df_dpr)
df_dpr_dataset = Dataset.from_dict(df_dpr)
df_dpr_dataset

Dataset({
    features: ['context'],
    num_rows: 5000
})

In [43]:
length = len(df_dpr_dataset)
length

5000

In [44]:
from tqdm import tqdm

questions = []
length = len(df_dpr_dataset)
for i in tqdm(range(length)):
# for i in tqdm(range(loops)):
  questions.append(_generate(df_dpr_dataset[i]['context']))

100%|██████████| 5000/5000 [23:01<00:00,  3.62it/s]


In [46]:
import json

df_dpr["question"] = questions
dpr_dataset_mt5_generated = Dataset.from_dict(df_dpr)
dpr_dataset_mt5_generated

# # data_dicts = final_mt5_data.to_dict()
# with open('/content/drive/MyDrive/IRE/datasets/dpr_dataset_mt5_generated.json', 'w') as fp:
#     json.dump(dpr_dataset_mt5_generated, fp, sort_keys=True, indent=4)

Dataset({
    features: ['context', 'question'],
    num_rows: 5000
})